## Look at the netscape file

In [ ]:
<!DOCTYPE NETSCAPE-Bookmark-file-1>
<!-- This is an automatically generated file.
     It will be read and overwritten.
     DO NOT EDIT! -->
<META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=UTF-8">
<TITLE>Bookmarks</TITLE>
<H1>Bookmarks</H1>
<DL><p># Ancestors
    <DT><H3 ADD_DATE="*" LAST_MODIFIED="*" PERSONAL_TOOLBAR_FOLDER="true">书签栏</H3> # Ancestors' names
    <DL><p>## Grandparents
        <DT><H3 ADD_DATE="*" LAST_MODIFIED="*">"##"</H3> ## Grandparents' names
        <DL><p>### Parents
            <DT><H3 ADD_DATE="*" LAST_MODIFIED="*">"###"</H3> ### Parent1's  name
            <DL><p>#### children
                <DT><A HREF="url1" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT1</A>
                <DT><A HREF="url2" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT2</A>
                <DT><A HREF="url3" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT3</A>
                <DT><A HREF="url4" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT4</A>
                <DT><A HREF="url5" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT5</A>
                .....
            <DT><H3 ADD_DATE="*" LAST_MODIFIED="*">"###"</H3> ### Parent2's  name
                <DT><A HREF="url1" ADD_DATE="*" ICON="data:image/png;base64,*">TEXT1</A>
            ......
            </DL><p
            .......  ### more Parents
        </DL><p>###
        ......   ## more Grandparents'
    </DL><p>##
    ......  # more Ancestors
</DL><p>#

## Code

In [48]:
from bs4 import BeautifulSoup
import requests
import re
from requests import RequestException

links = {}

def Fetching(url):
    s = requests.Session()
    try:
        r = s.get(url, allow_redirects=True)
        title = BeautifulSoup(r.text,"lxml").title
        if r.status_code == 200 and title:
            return title.string
        elif title is None:
            print('Request to ', url, "failed!!!", 'No title found!')
        else:
            if response.history:
                print("Request was redirected to ", r.url)
                rr = s.get(r.url)
                if rr.status_code == 200:
                    return BeautifulSoup(rr.text,"lxml").title.string
    except RequestException:
        print('Request to ', url, "failed!!!")

def bookmarks_to_md(Netscape_file, connection = True):
    for element in Netscape_file:
        if element.name == 'dt':
            folder_title = element.find('h3', recursive=False)
            if folder_title:
                if folder_title.parent.a is None:   ## empty folders
                    print(folder_title.string ,'is empty, has removed.')
                else:
                    f.write('\n###  ' + folder_title.string + "\n")
                    folder = element.dt.find('a', recursive=False)
                    if folder:
                        for a in element.find_all('a'):
                            if a['href'] in links:
                                links[a['href']].append(str(folder_title.string))
                                print("the %s has found in %s , its a duplicated link, it's removed" \
                                          %(a['href'],links[a['href']]))
                            else:
                                    links[a['href']]= [folder_title.string]
                                    if re.match("http://|https://",a.string): ## blank titles
                                        title =  re.sub(".*://.*?\.(.*?)\..*",'\\1',a['href'])
                                        if connection:
                                            print('Fetching For',a['href'],)
                                            temp_title = Fetching(a['href'])
                                            if temp_title is None:
                                                print('Rename the ',a.string, ' to ', title)
                                            else:
                                                print('Rename the ',a.string, ' to ', temp_title)
                                                title = temp_title
                                        else:
                                            print('Rename the ',a.string, ' to ', title)
                                        f.write("[{0}]({1})\n".format(title,a['href']))
                                    else:
        #                                 print(a.string)
        #                                 print(a['href'])
                                        if re.search("icon",str(a),re.S):
                                            try:
                                                f.write("[{0}]:{1}\n".format(a.string,a['icon']))
                                                f.write("![icon][{0}][{1}]({2})\n" .format(a.string,a.string,a['href']))
                                            except KeyError:
                                                print(a.string," loaded icon but encoutered a key error")
                                                f.write("[{0}]({1})\n".format(a.string,a['href']))
                                        else:
                                            f.write("[{0}]({1})\n".format(a.string,a['href']))
                    else:
                        return bookmarks_to_md(element,connection = connection)
            elif element.find('a', recursive=False):  ### isolate links
                s =element.find('a', recursive=False).string
                url = element.find('a', recursive=False)['href']
                f.write("[{0}]({1})\n".format(s,url))
#                 print(element.find('a', recursive=False).string)
#                 print(element.find('a', recursive=False)['href'])
        elif element.name=='a':  ## outside links
#             print(element.string)
#             print(element['href'])
              f.write("[{0}]({1})\n".format(element.string,element['href']))


def write_to_md(file,connection=True):
    with open(file,encoding='utf-8') as fl:
        soup = BeautifulSoup(fl,"html5lib").body.dt
        for p in soup.find_all('p'):
            p.unwrap()
        for dl in soup.find_all('dl'):
            dl.unwrap()
        # print(soup.prettify())
        fl.close()
        for title in soup.find_all('dt',recursive=False):
            if title.find("h3"):
                if title.a is None:   ## empty folders
                    print(title.h3.string ,'is empty,  has removed')
                else:
                    f.write("\n##  "  + title.find("h3").string)
                    f.write('\n---\n')
                    bookmarks_to_md(title,connection=connection)
       
with open("Bookmarks.md","w",encoding='utf-8') as f:        
    write_to_md("C:/Users/Lyole/Desktop/bookmarks_2018_3_17.html")
    print(len(links))
    f.close()

M is empty,  has removed
Ani is empty, has removed.
the http://game.ioxapp.com/ has found in ['小游戏', '小游戏'] , its a duplicated link, it's removed
the https://www.proprofs.com/games/ has found in ['小游戏', '小游戏'] , its a duplicated link, it's removed
the http://www.memagames.com/ has found in ['解谜', '解谜'] , its a duplicated link, it's removed
Fetching For https://bb.kunlun.com/deeplink/
Rename the  https://bb.kunlun.com/deeplink/  to  kunlun
the http://www.michaelbach.de/ot/index.html has found in ['奇葩', 'ins'] , its a duplicated link, it's removed
Fetching For http://free-line-design.com/
Rename the  http://free-line-design.com/  to  罫線・飾り罫ライン素材 FREE LINE DESIGN | 400点以上の罫線・ライン素材がフリー（無料）でダウンロードできる「FREE LINE DESIGN」。 すぐに使える商用利用可能なjpg、png、編集可能なイラレ(ベクター)データも用意してます。
FLAT ICON DESIGN -フラットアイコンデザイン- | フラットデザインに最適！WEBサイトやDTPですぐ使える商用利用可能なフラットアイコン素材がフリー（無料）ダウンロードできるサイト『FLAT ICON D  loaded icon but encoutered a key error
Fetching For http://event-pre.com/
Rename the  http://event-pre.com/  to  クリス